In [1]:
from dotenv import load_dotenv

_ = load_dotenv()
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"


from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file
openai_api_key = os.getenv("OPENAI_API_KEY")


from langchain.tools import tool
import uuid



from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
import sqlite3
from langchain_community.utilities import SQLDatabase
# from langchain_experimental.sql import SQLDatabaseChain

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('landlords.db')
cursor = conn.cursor()

# Create the table with an alphanumeric Contract_ID of at least 10 characters and an Email column
cursor.execute('''
CREATE TABLE IF NOT EXISTS address (
    Contract_ID TEXT,
    Date_of_Updation TEXT,
    Address TEXT,
    Email TEXT
)
''')

# Insert dummy data with alphanumeric Contract_ID (at least 10 characters) and unique emails for each Contract_ID
dummy_data = [
    ('A1B2C3D4E5', '2024-01-01', '123 Main St, Springfield', 'email1@example.com'),
    ('A1B2C3D4E5', '2024-02-01', '124 Main St, Springfield', 'email1@example.com'),
    ('F6G7H8I9J0', '2024-01-15', '456 Elm St, Shelbyville', 'email2@example.com'),
    ('F6G7H8I9J0', '2024-03-20', '457 Elm St, Shelbyville', 'email2@example.com'),
    ('K1L2M3N4O5', '2024-02-10', '789 Maple St, Capital City', 'email3@example.com'),
    ('P6Q7R8S9T0', '2024-04-05', '321 Oak St, Springfield', 'email4@example.com'),
    ('U1V2W3X4Y5', '2024-05-15', '654 Pine St, Shelbyville', 'email5@example.com'),
    ('Z1Y2X3W4V5', '2024-06-01', '987 Cedar St, Springfield', 'email6@example.com'),
    ('T6S5R4Q3P2', '2024-07-20', '741 Birch St, Shelbyville', 'email7@example.com'),
    ('N8M7L6K5J4', '2024-08-15', '852 Maple St, Capital City', 'email8@example.com'),
    ('I9H8G7F6E5', '2024-09-10', '963 Elm St, Springfield', 'email9@example.com'),
    ('D4C3B2A1Z0', '2024-10-05', '159 Oak St, Shelbyville', 'email10@example.com'),
    ('X1W2V3U4T5', '2024-11-15', '753 Pine St, Springfield', 'email11@example.com'),
    ('R5Q6P7O8N9', '2024-12-25', '246 Cedar St, Shelbyville', 'email12@example.com'),
    ('M3L2K1J0H9', '2025-01-30', '357 Birch St, Capital City', 'email13@example.com'),
    ('G8F7E6D5C4', '2025-02-14', '468 Maple St, Springfield', 'email14@example.com'),
    ('B2A1Z0Y9X8', '2025-03-10', '579 Elm St, Shelbyville', 'email15@example.com'),
    ('W3V4U5T6S7', '2025-04-05', '680 Oak St, Capital City', 'email16@example.com'),
    ('Q6P7O8N9M0', '2025-05-20', '791 Pine St, Springfield', 'email17@example.com')
]

# Insert data into the table
cursor.executemany('''
INSERT INTO address (Contract_ID, Date_of_Updation, Address, Email)
VALUES (?, ?, ?, ?)
''', dummy_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
# Function to query the table
def query_addresses(contract_id=None):
    # Connect to the SQLite database
    conn = sqlite3.connect('landlords.db')
    cursor = conn.cursor()

    if contract_id:
        # Query for a specific Contract_ID
        cursor.execute('SELECT * FROM address WHERE Contract_ID = ?', (contract_id,))
    else:
        # Query all records
        cursor.execute('SELECT * FROM address')
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Close the connection
    conn.close()
    
    return results

In [4]:
update_email_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
     ("system", "You are a helpful and polite landlord portal support agent. Your task is to assist users with updating their email addresses. Follow these guidelines strictly:\n"
               "1. If the user wants to update the email address, prompt the user to provide the Contract ID.\n"
               "2. Replace the email address for the contract ID in the database.\n"
               "3. Do not ask for any other information.\n"
               "4. Do not answer any questions outside of updating the email addresses.\n"
               "5. If the user asks about the conversation so far, answer within the context.\n"
               "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
               "7. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
               "8. If the user wants to go back one step, provide the previous stage's information.\n"
               "9. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
               "\n"
               "Example Conversations:\n"
               "\nExample 1:\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your contract ID?\n"
               "User: It's 123456.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is newuser@example.com.\n"
               "Assistant: Thank you! I’ve updated your email address to newuser@example.com. You should receive a confirmation email shortly. Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 2:\n"
               "User: I want to change my email address on my account.\n"
               "Assistant: No problem! Could you please provide your contract ID?\n"
               "User: It's 123456.\n"
               "Assistant: Great, thanks! Could you also provide the new email address you’d like to use?\n"
               "User: Sure, the new email is newuser@example.com.\n"
               "Assistant: Thank you for the information. Your email address has been successfully updated to newuser@example.com. You’ll receive a confirmation email shortly. Is there anything else you need help with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a wonderful day!\n"
               "\nExample 3:\n"
               "User: How can I update my email address on your system?\n"
               "Assistant: I can assist you with that! Please provide your contract ID.\n"
               "User: It's 123456.\n"
               "Assistant: Thank you! Now, could you provide the new email address you want to use?\n"
               "User: My new email is newuser@example.com.\n"
               "Assistant: Great, I’ve updated your email to newuser@example.com. A confirmation email will be sent to your new address. Is there anything else I can do for you?\n"
               "User: No, that's it. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 4 (User doesn't provide contract ID initially):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your contract ID?\n"
               "User: Do I really need to provide that?\n"
               "Assistant: Yes, for security reasons, we need your contract ID to process the update. Could you please provide it?\n"
               "User: Okay, my contract ID is 123456.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is newuser@example.com.\n"
               "Assistant: Thank you! I’ve updated your email address to newuser@example.com. You should receive a confirmation email shortly. Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 5 (User doesn't provide new email address initially):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your contract ID?\n"
               "User: My contract ID is 123456.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: I forgot to mention, it's newuser@example.com.\n"
               "Assistant: Thank you! I’ve updated your email address to newuser@example.com. You should receive a confirmation email shortly. Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 6 (User provides new email address and contract ID initially):\n"
               "User: Hi, I need to change my email address to newuser@example.com. My contract ID is 123456.\n"
               "Assistant: Thank you! I’ve updated your email address to newuser@example.com. You should receive a confirmation email shortly. Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"),
    ("user", "{user_input}")
])

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field
from sqlite3 import connect, ProgrammingError
from langchain.tools import tool

class EmailDetails(BaseModel):
    """Extracts Entities such as Contract ID and Email ID from the user input."""
    contract_id: str = Field(description="Contract ID")
    email_id: str = Field(description="Email ID")
        
        

@tool(args_schema=EmailDetails)
def update_email_knowledge(contract_id: str, email_id: str) -> str:
    """
    Handles updating the email address for a given contract ID and returns a status message.
    """
    # Reconnect to the existing SQLite database
    conn = connect('landlords.db')
    cursor = conn.cursor()

    # Fixed table name
    table_name = "address"

    # Prepare SQL for updating email address based on contract ID
    sql = f"UPDATE {table_name} SET Email = ? WHERE Contract_ID = ?"

    try:
        # Execute the update query
        cursor.execute(sql, (email_id, contract_id))
        # Commit the changes
        conn.commit()
        result_message = "Email address updated successfully."
    except ProgrammingError as e:
        result_message = f"Error with update operation: {e}"
    finally:
        # Close the connection
        conn.close()
    
    return result_message


In [7]:
update_email_knowledge.invoke({"contract_id":'A1B2C3D4E5' , "email_id": 'email_updated@example.com'})

'Email address updated successfully.'

In [8]:
from pydantic import BaseModel, Field, EmailStr
from email.message import EmailMessage
import ssl
import smtplib
import logging
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO)

@tool(args_schema=EmailDetails)
def send_update_email(contract_id: str, email_id: str) -> str:
    """
    Sends an email to the user's new email id  with a reference ID for  update confirmation.
    
    Args:
    receiver_email (str): The email address of the receiver.
    
    Returns:
    str: A message indicating the success or failure of the email sending process.
    
    Example:
    >>> send_update_email("receiver@example.com")
    'Email sent successfully.'
    """
    # Generate a unique reference ID
    reference_id = str(uuid.uuid4())
    
    # SMTP configuration
    smtp_server = "smtp.gmail.com"
    smtp_port = 465
    smtp_username = "chatbot.testing.05@gmail.com"
    smtp_password = "dlwg ovip zkso dyow"  # app-specific password

    
    # Email content
    subject = 'LLP- Address Update'
    body = f"""\
    We have received your request for updating the address for your contract ID. Kindly find your reference ID for further communications.

    Reference ID: {reference_id}

    Regards,
    Landlord portal
    """
    
    instance = EmailMessage()
    instance["From"] = smtp_username
    instance["To"] = email_id
    instance["Subject"] = subject
    instance.set_content(body)
    
    context = ssl.create_default_context()

    try:
        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as smtp:
            smtp.login(smtp_username, smtp_password)
            smtp.sendmail(smtp_username, email_id , instance.as_string())
            logging.info(f"Email sent successfully to {email_id} with reference ID {reference_id}.")
            return "Email sent successfully."
    except Exception as e:
        logging.error(f"Failed to send email to {email_id} with reference ID {reference_id}. Error: {e}")
        return f"Failed to send email: {e}"

In [9]:
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "update_email_knowledge": update_email_knowledge, 
            "send_update_email": send_update_email
        }
        return tools[result.tool].run(result.tool_input)

In [11]:
from langchain_core.utils.function_calling import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor

tools = [update_email_knowledge, send_update_email]
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
output_parser = OpenAIFunctionsAgentOutputParser()

update_email_address_chain = update_email_address_prompt | model | output_parser

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
agent_executor_chain = AgentExecutor(agent=update_email_address_chain,tools = tools,verbose=True, memory=memory)
#agent_executor_chain.invoke({"user_input": 'HI'})

In [ ]:
while True:
    user_input = input()
    if user_input in ['exit','bye']:
        break
    else:
        response = agent_executor_chain.invoke({
        "user_input": user_input
        })
        message = ""


change me email id to aplas@gmails.com and my contract id is adcfg123478


> Entering new AgentExecutor chain...
Sure! I've updated your email address to aplas@gmails.com for contract ID adcfg123478. You should receive a confirmation email shortly. Is there anything else I can assist you with?

> Finished chain.
what is my contract id ?


> Entering new AgentExecutor chain...
To update your email address, I'll need your contract ID. Could you please provide me with your contract ID?

> Finished chain.
what is the contract id i have provided 


> Entering new AgentExecutor chain...
To update your email address, I'll need your contract ID. Could you please provide me with your contract ID?

> Finished chain.


In [21]:
result = update_email_address_chain.invoke({"user_input": "Hi"})

In [22]:
result

'Hello! How can I assist you today?'

In [10]:
 tools = {
            "update_email_knowledge": update_email_knowledge, 
        }

In [ ]:
result.tool

In [25]:
result.tool_input

{'contract_id': 'U1V2W3X4Y5', 'email_id': 'email_update@example.com'}

In [26]:
tools[result.tool].run(result.tool_input)

'Email address updated successfully.'

In [32]:
# Example usage
print("All records:")
for row in query_addresses():
    print(row)

print("\nRecords for Contract_ID U1V2W3X4Y5:")
for row in query_addresses("K1L2M3N4O5"):
    print(row)

All records:
('A1B2C3D4E5', '2024-01-01', '123 Main St, Springfield', 'email_updated@example.com')
('A1B2C3D4E5', '2024-02-01', '124 Main St, Springfield', 'email_updated@example.com')
('F6G7H8I9J0', '2024-01-15', '456 Elm St, Shelbyville', 'email2@example.com')
('F6G7H8I9J0', '2024-03-20', '457 Elm St, Shelbyville', 'email2@example.com')
('K1L2M3N4O5', '2024-02-10', '789 Maple St, Capital City', 'email_update@example.com')
('P6Q7R8S9T0', '2024-04-05', '321 Oak St, Springfield', 'email4@example.com')
('U1V2W3X4Y5', '2024-05-15', '654 Pine St, Shelbyville', 'email_update@example.com')
('Z1Y2X3W4V5', '2024-06-01', '987 Cedar St, Springfield', 'email6@example.com')
('T6S5R4Q3P2', '2024-07-20', '741 Birch St, Shelbyville', 'email7@example.com')
('N8M7L6K5J4', '2024-08-15', '852 Maple St, Capital City', 'email8@example.com')
('I9H8G7F6E5', '2024-09-10', '963 Elm St, Springfield', 'email9@example.com')
('D4C3B2A1Z0', '2024-10-05', '159 Oak St, Shelbyville', 'email10@example.com')
('X1W2V3U4T